In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np

In [2]:
tf.keras.backend.clear_session()

In [3]:
SEQUENCE_LENGTH = 60
LSTM_SIZE = 128

In [4]:
embedding_matrix = np.load('./sentiment140_embedding.npy')

In [5]:
# not-exists word

embedding_matrix = np.pad(embedding_matrix, ((0,0),(0,1)), 'constant', constant_values=0)
embedding_matrix[-1][-1] = 1

In [6]:
tf.keras.backend.clear_session()
inputs = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH,))
x = tf.keras.layers.Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False,mask_zero=True)(inputs)
x = tf.keras.layers.SpatialDropout1D(0.2)(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(LSTM_SIZE, return_sequences=True))(x)
x = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=200)(x, x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs, name='sentiment140_init_model')
model.summary()

2022-03-20 16:15:56.766447: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-20 16:15:56.766562: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Model: "sentiment140_init_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 60, 301)      8153187     ['input_1[0][0]']                
                                                                                                  
 spatial_dropout1d (SpatialDrop  (None, 60, 301)     0           ['embedding[0][0]']              
 out1D)                                                                                           
                                                                                                  
 bidirectional (Bidirectional)  (None, 60, 256

In [7]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer=tfa.optimizers.AdamW(weight_decay=1e-4),
    metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5)]
)

In [8]:
model.save('./sentiment140_init_model.h5')